# ETL
This project extract real estate properties from an API, transform and loads them into a CSV.

Extract    
Extract all properties from Immo API. Call API.

Transform   
Transform (normalize) the nested jsons. Fill NAN values.
 
Load   
Load finish csv into S3 Bucket.

In [1]:
# Importing modules
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.mode.chained_assignment = None

DEBUG = True

In [2]:
import sys
sys.path.insert(1, '../pipeline')
sys.path.insert(2, '../data')

from etl_staging.etl_staging import ETL
# This is how you start the pipeline
etl = ETL()
# etl.execute("APARTMENTBUY")
etl.execute("HOUSEBUY")

INFO: Execute for category: HOUSEBUY
__________________________________________________
INFO: Start extracting data
INFO: Type: HOUSEBUY, size: 300, offset: 0
INFO: Property url: https://api.thinkimmo.com/immo?active=true&type=HOUSEBUY&sortBy=publishDate,desc&from=0&size=300&grossReturnAnd=false&allowUnknown=false&ownCapital=10&ownCapitalInPercent=true&ownCapitalAdditionalCost=false&managementCostInPercent=true&renovationCostInPercent=true&renovationCost=0&interestRate=1.5&repaymentRate=2&managementCost=35&additionalPurchaseCost=2&favorite=false&excludedFields=true&geoSearches=[]&averageAggregation=buyingPrice%3BpricePerSqm%3BsquareMeter%3BconstructionYear%3BrentPrice%3BrentPricePerSqm%3BrentPricePerSqm%3BrunningTime
INFO: Get API data batch 0-300 of total: 600
<Response [200]>
SUCCESS: Request OK
INFO: Response data 89015 found
INFO: Type: HOUSEBUY, size: 300, offset: 300
INFO: Property url: https://api.thinkimmo.com/immo?active=true&type=HOUSEBUY&sortBy=publishDate,desc&from=300&size

## Transformation 
We want to clean the nested JSON (normalize them). 

In [100]:
import json
# package for flattening json in pandas df
from pandas.io.json import json_normalize 

JSON_PATH = "../data/immo.json"

with open(JSON_PATH) as json_file:
    data = json.load(json_file)

def transform(json_raw):
    # normalize (tranform nested jsons to multiple columns) to dataframe
    df_raw = json_normalize(json_raw)
    # flatten nested json arrays for 'platforms' and 'buyingPriceHistory' 
    df_flat = df_raw.assign(platforms=df_raw['platforms']).explode('platforms')
    df_flat = df_flat.assign(buyingPriceHistory=df_raw['buyingPriceHistory']).explode('buyingPriceHistory')
    # again transform nested json columns to clean dataframe
    df_normalized = json_normalize(df_flat.to_dict('records'))
    return df_normalized

transform(data).info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 163 columns):
 #    Column                                   Dtype  
---   ------                                   -----  
 0    id                                       object 
 1    title                                    object 
 2    zip                                      object 
 3    buyingPrice                              float64
 4    rooms                                    float64
 5    squareMeter                              float64
 6    comission                                float64
 7    rentPricePerSqm                          float64
 8    pricePerSqm                              float64
 9    rentPrice                                float64
 10   rentPriceCurrent                         float64
 11   rentPriceCurrentPerSqm                   float64
 12   region                                   object 
 13   foreClosure                              bool   
 14   grossRet

Here we describe the process on transformation 

In [82]:
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df

In [83]:

with open(JSON_PATH) as json_file:
    data = json.load(json_file)
df = pd.DataFrame.from_dict(data)
df.head(3)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,platforms,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,address,region,foreClosure,locationFactor,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,aggregations,leasehold,priceInMarket,oAddress,originalAddress,houseMoney,images,buyingPriceHistory,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages
0,457f0ced1570618f7355f104101899ef,Jetzt schnell sein und mitgestalten - Penthouse,60599,789000.0,3.0,129.00,5.81,"[{'name': 'ebk', 'id': 'ebk04dc6b3c7e1b6d06dcd...",16.046512,6116.279070,2070.0,NaN,NaN,"{'ISO_3166-1_alpha-2': 'DE', 'ISO_3166-1_alpha...",Hessen,False,"{'population': 763380, 'populationTrend': {'fr...",3.15,NaN,2021.0,TERRACED_FLAT,FIRST_TIME_USE,NaN,False,NaN,NaN,False,False,True,False,2021-02-19T17:01:25.142Z,False,"{'district': {'name': 'Süd', 'buyingPrice': 63...",False,85.297297,"{'ebk': {'street': None, 'postcode': '60599', ...","{'street': '', 'postcode': '60599', 'locationQ...",NaN,[],"[{'buyingPrice': 789000, 'platformName': 'ebk'...",False,False,0,2021-02-19T17:01:25.431Z,0,None,-1043.25,-15.866920,-1043.25,False
1,febc9045602be17b45272286419f9533,Fußläufig in die Innenstadt! Attraktive 71m² g...,59368,195000.0,2.0,71.08,2.99,"[{'name': 'is24', 'id': '126734904', 'url': 'h...",6.021384,2743.387732,428.0,NaN,NaN,"{'ISO_3166-1_alpha-2': 'DE', 'ISO_3166-1_alpha...",Nordrhein-Westfalen,False,"{'population': 29717, 'populationTrend': {'fro...",2.63,NaN,1927.0,GROUND_FLOOR,WELL_KEPT,2012.0,False,NaN,2.0,True,False,True,False,2021-02-19T17:01:05.000Z,False,"{'location': {'name': 'Werne', 'buyingPrice': ...",False,189.320388,"{'is24': {'street': None, 'postcode': '59368',...","{'postcode': '59368', 'location': 'Werne'}",195.0,"[{'id': '28262858476863481ee67dbc5ffb4df8', 'o...","[{'buyingPrice': 195000, 'platformName': 'is24...",False,False,0,2021-02-19T17:04:26.608Z,0,None,-299.30,-18.418462,-299.30,True
2,f309110164a043d2db71b7027a40f673,Sonnige 2-Zimmer-Dachgeschosswohnung mit Westb...,85622,283000.0,2.0,42.25,2.38,"[{'name': 'is24', 'id': '126734881', 'url': 'h...",17.065089,6698.224852,721.0,NaN,NaN,"{'ISO_3166-1_alpha-2': 'DE', 'ISO_3166-1_alpha...",Bayern,False,"{'population': 0, 'populationTrend': {'from': ...",3.06,NaN,1999.0,ROOF_STOREY,FULLY_RENOVATED,NaN,True,3.0,3.0,True,False,True,False,2021-02-19T16:59:36.000Z,False,"{'location': {'name': 'Feldkirchen', 'buyingPr...",False,96.587031,"{'is24': {'street': None, 'postcode': '85622',...","{'postcode': '85622', 'location': 'Feldkirchen'}",157.0,"[{'id': 'bf954a438a27ced66e6df75067eb1972', 'o...","[{'buyingPrice': 283000, 'platformName': 'is24...",False,False,0,2021-02-19T17:04:30.274Z,0,None,-339.27,-14.385866,-339.27,True


### Inspection of the data

3 Sub rows   
Goal: We have the data (no loss of data)   
Case 1 Duplicate expose ID (because of platforms)   --> Our current go to strategy   
Case 2 Single expose ID (nested platforms)          --> harder to parse and display  
Case 3 Duplicate Expose ID (duplicate platforms)    -->   
    
platforms, buyingPriceHistory have nested array  

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      600 non-null    object 
 1   title                   600 non-null    object 
 2   zip                     599 non-null    object 
 3   buyingPrice             571 non-null    float64
 4   rooms                   574 non-null    float64
 5   squareMeter             591 non-null    float64
 6   comission               303 non-null    float64
 7   platforms               600 non-null    object 
 8   rentPricePerSqm         600 non-null    float64
 9   pricePerSqm             568 non-null    float64
 10  rentPrice               591 non-null    float64
 11  rentPriceCurrent        71 non-null     float64
 12  rentPriceCurrentPerSqm  71 non-null     float64
 13  address                 600 non-null    object 
 14  region                  600 non-null    ob

### Normalize nested arrray structures of jsons and nested jsons


Things that are always given are id, title, zip.

We have to be careful with the purchase price. There are missing values that have to be filled or kicked out. 

The json array has nested jsons/arrays. In the next step we try to flaten the json columns and aggregate them to one. 

The columns are:
- platforms
- buyingPriceHistory
- locationFactor
- aggregations
- oAddress
- originalAddress

In [85]:
df_raw = json_normalize(data)
df_raw.head(2)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,platforms,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,buyingPriceHistory,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.suburb,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.street,originalAddress.postcode,originalAddress.locationQuery,address.county,address.town,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,originalAddress.location,address.village,oAddress.is24.district,originalAddress.district,address.house_number,address.landuse,address.local_administrative_area,address.borough,address.road,oAddress.ebk.location,address.municipality,address.residential,address.railway,address.neighbourhood,address.office,address.highway,address.road_type,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ivd24.district,oAddress.wohnungjetzt.street,oAddress.wohnungjetzt.postcode,oAddress.wohnungjetzt.location,address.building,address.community_centre,oAddress.immobilien.street,oAddress.immobilien.postcode,oAddress.immobilien.location,address.shop,address.retail,address.fuel,address.craft,address.hamlet,address.quarter,address.boundary,oAddress.regionalimmobilien.street,oAddress.regionalimmobilien.postcode,oAddress.regionalimmobilien.location,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.place_of_worship,address.information,address.hospital,address.industrial,address.attraction,oAddress.ohnemakler.street,oAddress.ohnemakler.postcode,oAddress.ohnemakler.location,address.city_block,address.hotel,address.waterway,address.leisure,address.cafe
0,457f0ced1570618f7355f104101899ef,Jetzt schnell sein und mitgestalten - Penthouse,60599,789000.0,3.0,129.00,5.81,"[{'name': 'ebk', 'id': 'ebk04dc6b3c7e1b6d06dcd...",16.046512,6116.279070,2070.0,NaN,NaN,Hessen,False,3.15,NaN,2021.0,TERRACED_FLAT,FIRST_TIME_USE,NaN,False,NaN,NaN,False,False,True,False,2021-02-19T17:01:25.142Z,False,False,85.297297,NaN,[],"[{'buyingPrice': 789000, 'platformName': 'ebk'...",False,False,0,2021-02-19T17:01:25.431Z,0,None,-1043.25,-15.866920,-1043.25,False,DE,DEU,postcode,postcode,Frankfurt am Main,Süd,Europe,Deutschland,de,European Union,60599,Hessen,HE,Sachsenhausen Süd,50.095928,8.710911,"60599 Frankfurt am Main, Deutschland",763380,-5,5,True,7.3,66400.0,77,10.479452,25,25.00,16.664,Süd,638550.0,4950.0,3.175758,Frankfurt am Main,748200.0,5800,2.979310,None,925000.0,7170.542636,2.685405,

In [87]:
df_flat = df_raw.assign(platforms=df_raw['platforms']).explode('platforms')
df_flat = df_flat.assign(buyingPriceHistory=df_raw['buyingPriceHistory']).explode('buyingPriceHistory')
df_flat.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 674 entries, 0 to 599
Data columns (total 155 columns):
 #    Column                                   Dtype  
---   ------                                   -----  
 0    id                                       object 
 1    title                                    object 
 2    zip                                      object 
 3    buyingPrice                              float64
 4    rooms                                    float64
 5    squareMeter                              float64
 6    comission                                float64
 7    platforms                                object 
 8    rentPricePerSqm                          float64
 9    pricePerSqm                              float64
 10   rentPrice                                float64
 11   rentPriceCurrent                         float64
 12   rentPriceCurrentPerSqm                   float64
 13   region                                   object 
 14   foreClos

In [89]:
df_normalized = json_normalize(df_flat.to_dict('records'))
df_normalized.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 163 columns):
 #    Column                                   Dtype  
---   ------                                   -----  
 0    id                                       object 
 1    title                                    object 
 2    zip                                      object 
 3    buyingPrice                              float64
 4    rooms                                    float64
 5    squareMeter                              float64
 6    comission                                float64
 7    rentPricePerSqm                          float64
 8    pricePerSqm                              float64
 9    rentPrice                                float64
 10   rentPriceCurrent                         float64
 11   rentPriceCurrentPerSqm                   float64
 12   region                                   object 
 13   foreClosure                              bool   
 14   grossRet

In [90]:
df_normalized[:10]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.suburb,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.street,originalAddress.postcode,originalAddress.locationQuery,address.county,address.town,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,originalAddress.location,address.village,oAddress.is24.district,originalAddress.district,address.house_number,address.landuse,address.local_administrative_area,address.borough,address.road,oAddress.ebk.location,address.municipality,address.residential,address.railway,address.neighbourhood,address.office,address.highway,address.road_type,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ivd24.district,oAddress.wohnungjetzt.street,oAddress.wohnungjetzt.postcode,oAddress.wohnungjetzt.location,address.building,address.community_centre,oAddress.immobilien.street,oAddress.immobilien.postcode,oAddress.immobilien.location,address.shop,address.retail,address.fuel,address.craft,address.hamlet,address.quarter,address.boundary,oAddress.regionalimmobilien.street,oAddress.regionalimmobilien.postcode,oAddress.regionalimmobilien.location,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.place_of_worship,address.information,address.hospital,address.industrial,address.attraction,oAddress.ohnemakler.street,oAddress.ohnemakler.postcode,oAddress.ohnemakler.location,address.city_block,address.hotel,address.waterway,address.leisure,address.cafe,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory
0,457f0ced1570618f7355f104101899ef,Jetzt schnell sein und mitgestalten - Penthouse,60599,789000.0,3.0,129.00,5.81,16.046512,6116.279070,2070.0,NaN,NaN,Hessen,False,3.15,NaN,2021.0,TERRACED_FLAT,FIRST_TIME_USE,NaN,False,NaN,NaN,False,False,True,False,2021-02-19T17:01:25.142Z,False,False,85.297297,NaN,[],False,False,0,2021-02-19T17:01:25.431Z,0,None,-1043.25,-15.866920,-1043.25,False,DE,DEU,postcode,postcode,Frankfurt am Main,Süd,Europe,Deutschland,de,European Union,60599,Hessen,HE,Sachsenhausen Süd,50.095928,8.710911,"60599 Frankfurt am Main, Deutschland",763380,-5,5,True,7.3,66400.0,77,10.479452,25,25.000,16.664,Süd,638550.0,4950.0

In [49]:
df_test = df_raw.assign(platforms=df_raw['platforms']).explode('platforms')
df_test = df_test.assign(buyingPriceHistory=df_raw['buyingPriceHistory']).explode('buyingPriceHistory')
df_test[df['id'] == '3531fba10ec2058d1155a63a9e147c8d']


,id,title,zip,buyingPrice,rooms,squareMeter,comission,platforms,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,buyingPriceHistory,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.suburb,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.street,originalAddress.postcode,originalAddress.locationQuery,address.county,address.town,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,originalAddress.location,address.village,oAddress.is24.district,originalAddress.district,address.house_number,address.landuse,address.local_administrative_area,address.borough,address.road,oAddress.ebk.location,address.municipality,address.residential,address.railway,address.neighbourhood,address.office,address.highway,address.road_type,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ivd24.district,oAddress.wohnungjetzt.street,oAddress.wohnungjetzt.postcode,oAddress.wohnungjetzt.location,address.building,address.community_centre,oAddress.immobilien.street,oAddress.immobilien.postcode,oAddress.immobilien.location,address.shop,address.retail,address.fuel,address.craft,address.hamlet,address.quarter,address.boundary,oAddress.regionalimmobilien.street,oAddress.regionalimmobilien.postcode,oAddress.regionalimmobilien.location,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.place_of_worship,address.information,address.hospital,address.industrial,address.attraction,oAddress.ohnemakler.street,oAddress.ohnemakler.postcode,oAddress.ohnemakler.location,address.city_block,address.hotel,address.waterway,address.leisure,address.cafe
33,3531fba10ec2058d1155a63a9e147c8d,"Gartentraum in Bietigheim - große 3,5-Zimmerwo...",74321,380000.0,3.0,83.21,2.98,"{'name': 'ivd24', 'publishDate': '2021-02-19T1...",11.008292,4566.758803,916.0,NaN,NaN,Baden-Württemberg,False,2.89,NaN,1970.0,GROUND_FLOOR,WELL_KEPT,NaN,False,NaN,NaN,True,False,True,False,2021-02-19T16:35:08.000Z,False,False,122.977346,364.84,"[{'id': '570a38195aa28682596923141a30928c', 'o...","{'buyingPrice': 380000, 'platformName': 'is24'...",False,False,0,2021-02-19T16:37:25.804Z,0,None,-512.93,-16.197895,-512.93,True,DE,DEU,postcode,postcode,NaN,NaN,Europe,Deutschland,de,European Union,74321,Baden-Württemberg,BW,NaN,48.951549,9.122468,"74321 Bietigheim-Bissingen, Deutschland",43227,-5,5,False,3.8,0.0,53,20.131579,0,16.66,16.664,NaN,NaN,NaN,NaN,Bietigheim-Bissingen,316198.0,3800,3.568421,None,30

In [65]:
df_test[['id','platforms','buyingPriceHistory']][:3]

,id,platforms,buyingPriceHistory
33,3531fba10ec2058d1155a63a9e147c8d,"{'name': 'ivd24', 'publishDate': '2021-02-19T1...","{'buyingPrice': 380000, 'platformName': 'is24'..."
33,3531fba10ec2058d1155a63a9e147c8d,"{'name': 'is24', 'publishDate': '2021-02-19T16...","{'buyingPrice': 380000, 'platformName': 'is24'..."


In [75]:
df_normalized = json_normalize(df_test.to_dict('records'))
df_normalized.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 163 columns):
 #    Column                                   Dtype  
---   ------                                   -----  
 0    id                                       object 
 1    title                                    object 
 2    zip                                      object 
 3    buyingPrice                              float64
 4    rooms                                    float64
 5    squareMeter                              float64
 6    comission                                float64
 7    rentPricePerSqm                          float64
 8    pricePerSqm                              float64
 9    rentPrice                                float64
 10   rentPriceCurrent                         float64
 11   rentPriceCurrentPerSqm                   float64
 12   region                                   object 
 13   foreClosure                              bool   
 14   grossRet

In [ ]:
df_normalized[:10]

In [91]:
# entry with multiple platforms and same price history
df_normalized[df_normalized['id']=='3531fba10ec2058d1155a63a9e147c8d' ]

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.suburb,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.street,originalAddress.postcode,originalAddress.locationQuery,address.county,address.town,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,originalAddress.location,address.village,oAddress.is24.district,originalAddress.district,address.house_number,address.landuse,address.local_administrative_area,address.borough,address.road,oAddress.ebk.location,address.municipality,address.residential,address.railway,address.neighbourhood,address.office,address.highway,address.road_type,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ivd24.district,oAddress.wohnungjetzt.street,oAddress.wohnungjetzt.postcode,oAddress.wohnungjetzt.location,address.building,address.community_centre,oAddress.immobilien.street,oAddress.immobilien.postcode,oAddress.immobilien.location,address.shop,address.retail,address.fuel,address.craft,address.hamlet,address.quarter,address.boundary,oAddress.regionalimmobilien.street,oAddress.regionalimmobilien.postcode,oAddress.regionalimmobilien.location,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.place_of_worship,address.information,address.hospital,address.industrial,address.attraction,oAddress.ohnemakler.street,oAddress.ohnemakler.postcode,oAddress.ohnemakler.location,address.city_block,address.hotel,address.waterway,address.leisure,address.cafe,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory
33,3531fba10ec2058d1155a63a9e147c8d,"Gartentraum in Bietigheim - große 3,5-Zimmerwo...",74321,380000.0,3.0,83.21,2.98,11.008292,4566.758803,916.0,NaN,NaN,Baden-Württemberg,False,2.89,NaN,1970.0,GROUND_FLOOR,WELL_KEPT,NaN,False,NaN,NaN,True,False,True,False,2021-02-19T16:35:08.000Z,False,False,122.977346,364.84,"[{'id': '570a38195aa28682596923141a30928c', 'o...",False,False,0,2021-02-19T16:37:25.804Z,0,None,-512.93,-16.197895,-512.93,True,DE,DEU,postcode,postcode,NaN,NaN,Europe,Deutschland,de,European Union,74321,Baden-Württemberg,BW,NaN,48.951549,9.122468,"74321 Bietigheim-Bissingen, Deutschland",43227,-5,5,False,3.8,0.0,53,20.1315

## Inspect null values

In [92]:
df_normalized.isnull().sum()

id                                           0
title                                        0
zip                                          1
buyingPrice                                 29
rooms                                       26
squareMeter                                  9
comission                                  344
rentPricePerSqm                              0
pricePerSqm                                 32
rentPrice                                    9
rentPriceCurrent                           598
rentPriceCurrentPerSqm                     598
region                                       0
foreClosure                                  0
grossReturn                                 33
grossReturnCurrent                         598
constructionYear                           113
apartmentType                                0
condition                                    0
lastRefurbishment                          588
lift                                         0
floor        

In [93]:
df_normalized['zip'].unique().shape

(415,)

In [94]:
df_normalized[df_normalized['comission'].isnull()].sample(3)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,region,foreClosure,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,leasehold,priceInMarket,houseMoney,images,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages,address.ISO_3166-1_alpha-2,address.ISO_3166-1_alpha-3,address._category,address._type,address.city,address.city_district,address.continent,address.country,address.country_code,address.political_union,address.postcode,address.state,address.state_code,address.suburb,address.lat,address.lon,address.displayName,locationFactor.population,locationFactor.populationTrend.from,locationFactor.populationTrend.to,locationFactor.hasUniversity,locationFactor.unemploymentRate,locationFactor.numberOfStudents,locationFactor.score,locationFactor.unemploymentRateScore,locationFactor.universityScore,locationFactor.populationScore,locationFactor.populationTrendScore,aggregations.district.name,aggregations.district.buyingPrice,aggregations.district.pricePerSqm,aggregations.district.grossReturn,aggregations.location.name,aggregations.location.buyingPrice,aggregations.location.pricePerSqm,aggregations.location.grossReturn,aggregations.similarListing.name,aggregations.similarListing.buyingPrice,aggregations.similarListing.pricePerSqm,aggregations.similarListing.grossReturn,oAddress.ebk.street,oAddress.ebk.postcode,oAddress.ebk.locationQuery,originalAddress.street,originalAddress.postcode,originalAddress.locationQuery,address.county,address.town,oAddress.is24.street,oAddress.is24.postcode,oAddress.is24.location,originalAddress.location,address.village,oAddress.is24.district,originalAddress.district,address.house_number,address.landuse,address.local_administrative_area,address.borough,address.road,oAddress.ebk.location,address.municipality,address.residential,address.railway,address.neighbourhood,address.office,address.highway,address.road_type,oAddress.ivd24.street,oAddress.ivd24.postcode,oAddress.ivd24.location,oAddress.ivd24.district,oAddress.wohnungjetzt.street,oAddress.wohnungjetzt.postcode,oAddress.wohnungjetzt.location,address.building,address.community_centre,oAddress.immobilien.street,oAddress.immobilien.postcode,oAddress.immobilien.location,address.shop,address.retail,address.fuel,address.craft,address.hamlet,address.quarter,address.boundary,oAddress.regionalimmobilien.street,oAddress.regionalimmobilien.postcode,oAddress.regionalimmobilien.location,oAddress.wohnungsboerse.street,oAddress.wohnungsboerse.postcode,oAddress.wohnungsboerse.location,oAddress.wohnungsboerse.district,oAddress.immobilio.street,oAddress.immobilio.postcode,oAddress.immobilio.location,address.place_of_worship,address.information,address.hospital,address.industrial,address.attraction,oAddress.ohnemakler.street,oAddress.ohnemakler.postcode,oAddress.ohnemakler.location,address.city_block,address.hotel,address.waterway,address.leisure,address.cafe,platforms.name,platforms.id,platforms.url,platforms.creationDate,platforms.publishDate,platforms.active,buyingPriceHistory.buyingPrice,buyingPriceHistory.platformName,buyingPriceHistory.creationDate,buyingPriceHistory
654,22976e5602d28f81c5ace32006a558da,Helle 3-Zimmer-Wohnungen zum Kauf in Hagen,58089,79000.0,3.0,87.00,NaN,5.091954,908.045977,443.0,NaN,NaN,Nordrhein-Westfalen,False,6.73,NaN,1914.0,APARTMENT,NEED_OF_RENOVATION,NaN,False,2.0,4.0,False,False,True,False,2021-02-19T13:13:05.000Z,True,False,102.597403,NaN,"[{'id': '4107a7d8d27538a6ca5883f7a5d81fe2', 'o...",False,False,0,2021-02-19T15:30:30.924Z,0,None,60.45,9.182278,60.45,True,DE,DEU,building,building,Hagen,Hagen-Mitte,Europe,Deutschland,de,European Union,58089,Nordrhein-Westfalen,NW,Kuhlerkamp,51.354385,7.453917,"Minervastraße 1, 58089 Hagen, Deutschland",188686,-10,-5,True,12.2,679

In [95]:
# Check null values and unique values
df_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Columns: 163 entries, id to buyingPriceHistory
dtypes: bool(12), float64(41), int64(8), object(102)
memory usage: 803.1+ KB


In [99]:
# group by city
# group by type
df_normalized.groupby("zip")['cashFlow'].max().sort_values(ascending=False)[:10]

zip
70192    1816.10
67061     404.48
47798     311.13
39288     290.63
42107     269.58
67307     217.05
45127     212.55
42551     162.68
68167     154.37
06249     130.57
Name: cashFlow, dtype: float64

# Database
Here we want to save the CSV in the new creatd datbase schema.  

In [19]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_NAME  = os.getenv('DB_NAME')
DB_USER  = os.getenv('DB_USER')
DB_PASSWORD  = os.getenv('DB_PASSWORD')

connection_parameter = {
    "host"      : DB_HOST,
    "database"  : DB_NAME,
    "user"      : DB_USER,
    "password"  : DB_PASSWORD
}

def connect(connection_parameter):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**connection_parameter)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    # cursor = conn.cursor()
    # try:
    #     cursor.execute(select_query)
    # except (Exception, psycopg2.DatabaseError) as error:
    #     print("Error: %s" % error)
    #     cursor.close()
    #     return 1
    
    # # Naturally we get a list of tupples
    # tupples = cursor.fetchall()
    # cursor.close()
    print("Read from table")
    df = pd.read_sql_query(select_query,con=conn)
    
    # We just need to turn it into a pandas dataframe
    # df = pd.DataFrame(tupples, columns=column_names)
    return df

In [20]:
# Connect to the database
conn = connect(connection_parameter)
column_names = []#["id", "source", "datetime", "mean_temp"]
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "SELECT * FROM public.api_liveaiproperty")
df.head()

Connecting to the PostgreSQL database...
Connection successful
Read from table


,id,active,property_id,property_url,source_id,source_portal,property_type,image_urls,image_captions,city,title,address,rooms,living_space,floor,no_of_floors,construction_year,condition,latitude,longitude,zipcode,foreclosure,is_rented_out,created_at,updated_at,price_id
0,45857,True,6a867704a532a627f707d778d30d8439,https://www.immobilienscout24.de/expose/116894250,116894250,is24,flat,https://pictures.immobilienscout24.de/listings...,1AA1\n3ABD\n7\nLogopartner\nLogopartner,Berlin,HERRSCHAFTLICHE GARTENRESIDENZ mit Privatgarte...,"Tiergarten, Berlin, Deutschland",2.0,73.00,NaN,4.0,1880.0,FIRST_TIME_USE,52.509778,13.357260,10785,False,False,2020-12-15 15:59:06.680017+00:00,2020-12-15 15:59:06.684562+00:00,46854.0
1,33503,True,f65bb574710bc2bb043bea8069d10131,https://www.immobilienscout24.de/expose/120330418,120330418,is24,flat,https://pictures.immobilienscout24.de/listings...,Gartenperspektive\nInnenraumperspektive\nVogel...,Obersontheim,Exklusives 5-Zimmer-Penthouse mit Dachterrasse...,"Frankenstraße, 74423 Obersontheim, Deutschland",5.0,127.79,NaN,NaN,2021.0,FIRST_TIME_USE,49.058834,9.887907,74423,False,False,2020-12-13 15:34:46.452927+00:00,2020-12-15 15:38:05.368774+00:00,33650.0
2,33504,True,5835cc06d03607cc80f642d982de4eff,https://www.immobilienscout24.de/expose/120331198,120331198,is24,flat,https://pictures.immobilienscout24.de/listings...,Gartenperspektive\nInnenraumperspektive\nVogel...,Obersontheim,Raum & Lebensfreunde! Repräsentatives 5-Zimmer...,"Frankenstraße, 74423 Obersontheim, Deutschland",4.0,138.44,NaN,NaN,2021.0,FIRST_TIME_USE,49.058834,9.887907,74423,False,False,2020-12-13 15:34:46.464072+00:00,2020-12-15 15:38:05.374792+00:00,33651.0
3,34809,True,68e621da2bb7a3b7bb8557f04b74f3bd,https://www.immobilienscout24.de/expose/122422682,122422682,is24,flat,https://pictures.immobilienscout24.de/listings...,Diele\nHausflur\nKüche\nBadezimmer\nGäste-WC\n...,Düsseldorf,Sanierte und barrierefreie 4-Zimmer-Wohnung mi...,"Holthausen, 40589 Düsseldorf, Deutschland",4.0,90.00,NaN,NaN,1967.0,NO_INFORMATION,51.175156,6.832360,40589,False,False,2020-12-13 15:37:15.498003+00:00,2020-12-15 15:40:34.740057+00:00,34956.0
4,35827,True,4813f6cbf82e17c704360dcad5aa5480,https://www.immobilienscout24.de/expose/122184987,122184987,is24,flat,https://pictures.immobilienscout24.de/listings...,Wohnzimmer\nAußenansicht\nBadezimmer Beispiel\...,Nieste,Wohnen im Herzen von Nieste,"34329 Nieste, Deutschland",3.0,93.90,NaN,3.0,2020.0,FIRST_TIME_USE,51.312294,9.670994,34329,False,False,2020-12-13 15:39:42.735693+00:00,2020-12-15 15:43:02.089328+00:00,35974.0


In [21]:
df.shape

(108045, 26)

In [22]:
df.columns[df.dtypes == object]

Index(['property_id', 'property_url', 'source_id', 'source_portal',
       'property_type', 'image_urls', 'image_captions', 'city', 'title',
       'address', 'condition', 'zipcode', 'foreclosure', 'is_rented_out'],
      dtype='object')

In [23]:
#columns_strings = df.columns[df.dtypes == object]
columns_strings = ['property_id', 'property_url', 'source_id', 'source_portal',
       'property_type', 'image_urls', 'image_captions', 'city', 'title',
       'address', 'condition', 'zipcode']
for column in columns_strings:
    print(f"Column: {column}: {df[column].str.len().max()}")

Column: property_id: 32.0
Column: property_url: 288.0
Column: source_id: 50.0
Column: source_portal: 18.0
Column: property_type: 5
Column: image_urls: 6995.0
Column: image_captions: 1727.0
Column: city: 48.0
Column: title: 199.0
Column: address: 176.0
Column: condition: 34.0
Column: zipcode: 5.0


# ROI Calculation
This is part of the transformation. We will add more columns for the ROI/cashflow calculation and later use those methods to updte the ROI/cashflow on the fly for the API endpoints.

In [25]:
from transformer import get_roi_gross, get_cashflow_after_operating_expenses, get_cashflow_after_taxes, get_cashflow_after_reserves, get_tax_year


In [40]:
# FROM EXPOSE
purchase_price = 150000         # required
living_space_sqm = 60           # required
cold_rent_given = 700           # can be given or not; DEFAULT: None
house_allowance_given = 300     # can be given or not; DEFAULT: None


# OUR DEFAULT VALUES
rent_price_index_per_sqm = 11               # varies from city and living space
vacancy_rate = 0.02                         # 2% standard = no rent per year
house_allowance_rate = 0.35                 # 35% of cold rent
interest_rate = 0.02                        # 2% as standard
mortgage_rate = 0.02                        # 2% as standard
equity_rate = 0.2                           # 20% of purchase price 
maintenance_reserve_per_sqm_per_year = 10   # 10€ per sqm and per year as standard
tax_rate = 0.3                              # could be 20-30%
tax_write_off_rate = 0.02                   # 2% (has to be validated by professionals)

# VALUES FOR KEY METRICS CALCULATION
#   COLD RENT
cold_rent_month = cold_rent_given if cold_rent_given else cold_rent_estimated
cold_rent_year = cold_rent_month * 12

#   EQUITY
equity = purchase_price * equity_rate
#   LOAN
loan = purchase_price - equity

# ESTIMATED VALUES
cold_rent_estimated = living_space_sqm * rent_price_index_per_sqm
house_allowance_estimated = cold_rent_month * house_allowance_rate
maintenance_reserve_private_year = living_space_sqm * maintenance_reserve_per_sqm_per_year
maintenance_reserve_private_month = maintenance_reserve_private_year / 12
vacancy_year = cold_rent_year * vacancy_rate
vacancy_month = vacancy_year / 12
tax_write_off_year = tax_write_off_rate * loan
tax_write_off_month = tax_write_off_year / 12

#   HOUSE ALLOWANCE 
house_allowance_month = house_allowance_given if house_allowance_given else house_allowance_estimated
house_allowance_year = house_allowance_month * 12
#   INTEREST
interest_year = loan * interest_rate
interest_month = interest_year / 12
#   MORTGAGE
mortgage_year = loan * mortgage_rate
mortgage_month = mortgage_year / 12
#   HOUSE ALLOWANCE (Hausgeld)
tax = get_tax_year(cold_rent_year, vacancy_year, house_allowance_month, interest_year, tax_write_off_year, maintenance_reserve_private_year)

NameError: name 'get_tax_year' is not defined

In [41]:

roi_gross = get_roi_gross(cold_rent_year,purchase_price)

cashflow_after_operating_expenses = get_cashflow_after_operating_expenses(cold_rent_month, vacancy_month, house_allowance_month, interest_month, mortgage_month)

cashflow_after_taxes = get_cashflow_after_taxes(cold_rent_month, vacancy_month, house_allowance_month, interest_month, mortgage_month, tax)

cashflow_after_reserves = get_cashflow_after_reserves(cold_rent_month, vacancy_month, house_allowance_month, interest_month, mortgage_month, tax, maintenance_reserve_private_month)

print(roi_gross)
print(cashflow_after_operating_expenses)
print(cashflow_after_taxes)
print(cashflow_after_reserves)

NameError: name 'get_roi_gross' is not defined

In [42]:
df.sample(10)

,id,title,zip,buyingPrice,rooms,squareMeter,comission,platforms,rentPricePerSqm,pricePerSqm,rentPrice,rentPriceCurrent,rentPriceCurrentPerSqm,address,region,foreClosure,locationFactor,grossReturn,grossReturnCurrent,constructionYear,apartmentType,condition,lastRefurbishment,lift,floor,numberOfFloors,balcony,garden,active,rented,publishDate,privateOffer,aggregations,leasehold,priceInMarket,oAddress,originalAddress,houseMoney,images,buyingPriceHistory,priceReduced,priceIncreased,runningTime,lastUpdatedAt,favorite,favoriteDate,cashFlow,ownCapitalReturn,cashFlowPerLivingUnit,hasImages
478,5c2607457bdb53f2f05fb2eaed988743,Vermietete Einzimmerwohnung in zentraler Lage ...,68159,129000.0,1.0,36.00,1.95,[{'deactivationDate': '2021-02-19T09:01:08.690...,11.722222,3583.333333,422.0,310.000000,8.611111,"{'ISO_3166-1_alpha-2': 'DE', 'ISO_3166-1_alpha...",Baden-Württemberg,False,"{'population': 310658, 'populationTrend': {'fr...",3.93,2.88,1988.0,NO_INFORMATION,WELL_KEPT,NaN,True,7.0,7.0,False,False,True,True,2021-02-18T18:21:23.000Z,False,"{'district': {'name': 'Innenstadt/Jungbusch', ...",False,126.470588,"{'is24': {'street': None, 'postcode': '68159',...","{'postcode': '68159', 'location': 'Mannheim', ...",NaN,"[{'id': '81d34d244d9d038c57140ed89b51e533', 'o...","[{'buyingPrice': 129000, 'platformName': 'is24...",False,False,1,2021-02-18T18:24:19.755Z,0,None,-171.83,-15.984496,-171.83,True
107,ba8f834dc6d0c7deec3fd79bfe113bd7,Geräumige 3-Zimmer-Eigentumswohnung in MA-Waldhof,68305,85000.0,3.0,79.26,NaN,"[{'name': 'is24', 'id': '126717302', 'url': 'h...",10.017663,1072.419884,794.0,NaN,NaN,"{'ISO_3166-1_alpha-2': 'DE', 'ISO_3166-1_alpha...",Baden-Württemberg,True,"{'population': 310658, 'populationTrend': {'fr...",11.21,NaN,1900.0,APARTMENT,NEED_OF_RENOVATION,NaN,False,NaN,NaN,False,False,True,False,2021-02-19T08:09:04.000Z,False,"{'district': {'name': 'Waldhof', 'buyingPrice'...",False,34.274194,"{'is24': {'street': None, 'postcode': '68305',...","{'postcode': '68305', 'location': 'Mannheim', ...",NaN,"[{'id': 'd588b00303587dd4319af6fd4b650163', 'o...","[{'buyingPrice': 85000, 'platformName': 'is24'...",False,False,0,2021-02-19T08:12:33.302Z,0,None,276.93,39.096471,276.93,True
159,68527ade9cbb5d85e2580bc27eb51388,"🏡🌳GRÜN, RUHIG; ZENTRAL🌳: Neubau-Maisonette mit...",13156,941500.0,4.0,201.00,NaN,"[{'name': 'is24', 'id': '126714433', 'url': 'h...",15.522388,4684.079602,3120.0,NaN,NaN,"{'ISO_3166-1_alpha-2': 'DE', 'ISO_3166-1_alpha...",Berlin,False,"{'population': 3669491, 'populationTrend': {'f...",3.98,NaN,2021.0,NO_INFORMATION,FIRST_TIME_USE,2021.0,True,NaN,NaN,True,True,True,False,2021-02-19T06:52:16.000Z,False,"{'district': {'name': 'Niederschönhausen', 'bu...",False,78.132780,"{'is24': {'street': None, 'postcode': '13156',...","{'postcode': '13156', 'location': 'Berlin - Pa...",NaN,"[{'id': '71b1520678bdfd05aa3bdc74d465eafe', 'o...","[{'buyingPrice': 941500, 'platformName': 'is24...",False,False,0,2021-02-19T08:56:49.252Z,0,None,-664.08,-8.464153,-664.08,True
148,5e5e9e7b211959aa794428fd73d7e820,Moderne 2-Zimmer-Wohnung auf ca. 60 m² mit gro...,12529,287800.0,2.0,59.41,NaN,"[{'name': 'is24', 'id': '126668719', 'url': 'h...",12.943949,4844.302306,769.0,NaN,NaN,"{'ISO_3166-1_alpha-2': 'DE', 'ISO_3166-1_alpha...",Brandenburg,False,"{'population': 0, 'populationTrend': {'from': ...",3.21,NaN,2023.0,APARTMENT,FIRST_TIME_USE,NaN,True,6.0,NaN,True,False,True,False,2021-02-19T07:13:48.000Z,False,"{'district': {'name': 'Schönefeld', 'buyingPri...",False,96.254181,"{'is24': {'street': 'Rathausgasse 3', 'postcod...","{'street': 'Rathausgasse 3', 'postcode': '1252...",NaN,"[{'id': '3a28f1df5b796d153714bbf1cc7f04fc', 'o...","[{'buyingPrice': 287800, 'platformName': 'is24...",False,False,0,2021-02-19T09:17:24.653Z,0,None,-325.57,-13.574705,-325.57,True
229,8f3b9ec2069a33c6fd363b29a5717d8d,Großzügige Wohnmöglichkeit über zwei Ebenen...,22529,2175000.0,7.0,250.00,6.12,"[{'name': 'ebk', 'id': 'ebkf295daea30d13c350dc...",15.252000,8700.000000,3813.0,N

In [43]:
df_cashflow_comparisson = df[["id","title","zip","buyingPrice","cashFlow"]] 
df_cashflow_comparisson["liveAiCashflow"] = get_cashflow_after_operating_expenses(df_cashflow_comparisson["buyingPrice"],)
df_cashflow_comparisson.head()

NameError: name 'get_cashflow_after_operating_expenses' is not defined

# Next steps

- Clean null values. 
- Transform into clean schema.
- Load clean csv into S3 bucket.
